# Animate time series data

In this tutorial, I explain how to animate traces from patch-clamp recordings. The full tutorial for this notebook can be found in [Patch-clamp data analysis in Python: animate time series data](https://spikesandbursts.wordpress.com/2024/01/04/patch-clamp-data-analysis-animate-time-series/) of the [Spikes and Bursts](https://spikesandbursts.wordpress.com/) blog.

Interactive plots in Jupyter lab using `%matplotlib widget` ([Magic matplotlib](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib))

Libraries specific to this notebook:
* [pyABF](https://github.com/swharden/pyABF)
* [MoviePy](https://zulko.github.io/moviepy/)
* [Scipy wavfile](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html).

# Import the libraries

In [ ]:
# Libraries for numpy arrays and data tables
import numpy as np
import pandas as pd
import os

# Import abf files
import pyabf

# Find Peaks and audio functions
import scipy
from scipy import signal  # Filtering
from scipy.signal import find_peaks  # Find Peaks function
from scipy.io.wavfile import write  # Audio from NumPy array

# Plots and animations
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import animation

# Merge video and audio files
from moviepy.editor import VideoFileClip, AudioFileClip

# Interactive plots in Jupyter lab
%matplotlib widget  
plt.close('all')

# Create paths

In [ ]:
notebook_name = 'time-series_animations'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data':  f'{data_path}/Data',
         'processed_data': f'{data_path}/Processed_data/{notebook_name}',
         'analysis': f'{data_path}/Analysis/{notebook_name}'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Load data

Examples files:
- ABF: **pfc_lhx6_cell-attached.abf** and **mesc_nkx2_aps.abf**
- TXT: **pfc_lhx6_cell-attached.txt**

## ABF files

In [ ]:
# Comment out this cell to check and load the raw .txt data file

# ABF file/s
recording = "pfc_lhx6_cell-attached"  # or "pfc_lhx6_cell-attached"

data_path = f"{paths['data']}/{recording}.abf" 
abf = pyabf.ABF(data_path)
print(abf)

# Sampling rate
fs = int(abf.dataPointsPerMs * 1000)

# Quick plot to see the trace/s
plt.figure(figsize=(6,3))

# To select channel/sweep: abf.setSweep(sweepNumber=0, channel=0)
for sweepNumber in abf.sweepList:  # Only 1 sweep in the example
    abf.setSweep(sweepNumber)
    y_variable = abf.sweepY
    time = abf.sweepX
    
    # Plot
    plt.plot(time, y_variable)
    plt.ylabel(abf.sweepLabelY)
    plt.xlabel(abf.sweepLabelX)

# Print the sampling rate
print("Sampling rate:", fs)

# Show the plot
plt.tight_layout()  # Adjust the padding around the plot
plt.show()

## TXT files

In [ ]:
# # Uncomment this cell to check and load the raw .txt data file
# recording = "pfc_lhx6_cell-attached" 

# # Load the text file (note: uses tab '\t' as the delimiter)
# df = pd.read_csv(f"{paths['data']}/{recording}.txt", delimiter="\t")

# # Extract columns
# time = df.iloc[:, 0] 
# y_variable = df.iloc[:, 1]

# # Convert to NumPy arrays
# time = time.to_numpy()
# y_variable = y_variable.to_numpy()

# Filter the signal

In [ ]:
# Lowpass Bessel filter
b_lowpass, a_lowpass = signal.bessel(4,     # Order of the filter
                                     2000,  # Cutoff frequency
                                     'low', # Type of filter
                                     analog=False,  # Analog or digital filter
                                     norm='phase',  # Critical frequency normalization
                                     fs=fs)  # fs: sampling frequency

signal_filtered = signal.filtfilt(b_lowpass, a_lowpass, y_variable)

# Adjust the baseline if needed
# signal_filtered = signal_filtered - np.median(signal_filtered)

# Simple plot
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(time, signal_filtered)
plt.xlabel(abf.sweepLabelX)
plt.ylabel(abf.sweepLabelY)

# Show the plot
plt.tight_layout()
plt.show()

# Find peaks

In [ ]:
# Assign the variables here to simplify the code
# time = abf.sweepX
y_variable = signal_filtered  # Raw or filtered signal

# Threshold for peak detection (absolute value)
peaks_theshold = 50

# Find peaks function
peaks, peaks_dict = find_peaks(-y_variable,  # Note: change the polarity of signal for negative peaks
                               height=peaks_theshold)

# Plot the detected spikes in the trace
fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(time, y_variable)

# Red dot for each detected spike
ax.plot(peaks/fs, y_variable[peaks], "r.")
ax.set_xlabel(abf.sweepLabelX)
ax.set_ylabel(abf.sweepLabelY)

fig.tight_layout()  # Adjust layout
plt.show()

# Animate the trace

In [ ]:
# Initialize the plot
fig, ax = plt.subplots(figsize=(8, 3)) 

# Define plot parameters
line, = ax.plot([], [], lw=1, color='tab:blue')  # Initialize a line plot
events, = ax.plot([], [], 'o', color='magenta', markersize=3)  # Optional: detected events

# Axis options
ax.set_xlabel(abf.sweepLabelX)  # Set x-axis label
ax.set_ylabel("Current (pA)")  # You can also use 'abf.sweepLabelY'
ax.set_xlim(np.min(time), np.max(time))  # Set x-axis limits 

# Set y-axis limits with some upper and bottom blank space
ax.set_ylim(np.min(y_variable) + (0.15 * np.min(y_variable)), 
            np.max(y_variable) + (0.1 * np.max(y_variable)))  

# Animation function
interval = 400  # Adapt to your data fs

def animate(frame):
    end_frame = (frame + 1) * interval  # End frame for each update in the animation
    line.set_data(time[:end_frame], y_variable[:end_frame])  # Update line plot data

    # Comment out the 'events' lines if you do not want to show detected peaks
    events_time = time[peaks[peaks < end_frame]]  # Extract event times
    events_signal = y_variable[peaks[peaks < end_frame]]
    events.set_data([events_time], [np.min(y_variable[peaks]) * 1.05])  # Event dots at fixed height
    # events.set_data(events_time, events_signal * 1.1)  # Event dots at variable heights
    
    return line, events

# Create the animation
frames = len(time) // interval

anim = animation.FuncAnimation(fig, animate, frames=frames,
                               interval=5, blit=True, repeat=False)

print("Animation (number of frames):", frames)

fig.tight_layout()  # Adjust layout
plt.show()  # Display the animation

# Save the animated trace

## Save as mp4

In [ ]:
%%time

# Animation parameters
duration_s = 10  # Select the length of the video here

anim_fps = frames/duration_s

# Save as mp4 
extra_args = [
    '-vcodec', 'libx264',  # Video codec
    '-s', '2400x900',]  # Set the output resolution, same ratio as figure size

# Create the video with FFMpegWriter
writer = animation.FFMpegWriter(fps=anim_fps, bitrate=3000, 
                                codec="h264",  extra_args=extra_args)

# Save path
anim.save(f"{paths['analysis']}/{recording}_video.mp4", writer=writer)

# Print the fps and duration of the final video
print('Video_duration (s):', frames/anim_fps)
print('Video_fps:', anim_fps)

## Save as gif

In [ ]:
%%time 

writergif = animation.PillowWriter(fps=anim_fps, bitrate=2000)

anim.save(f"{paths['analysis']}/{recording}_video.gif", writer=writergif)

# Create audio

* [Scipy wavfile](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html)
* [Soundfile](https://pysoundfile.readthedocs.io/en/latest/)
* [IPython display](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html)

In [ ]:
# Audio parameters
amplitude = np.iinfo(np.int16).max  # Get the max value of the audio type (e.g. 16 bits)
audio_fs = int(len(y_variable)/duration_s)
gain = 1  # Set to 1 if not needed

# Normalize the ephys data in the range [-1, 1]
normalized_data = y_variable / np.max(np.abs(y_variable))

# Scale data option 1: to the audio range and, optionally, multiply by some gain
# scaled_data = np.int16(normalized_data * amplitude * gain)

# Scale data option 2: clipping the data to get less background noise
scaled_data = np.int16(np.clip(normalized_data, -1, -0.06) * amplitude * gain)

# Save the NumPy array as a WAV file
write(f"{paths['analysis']}/{recording}_audio.wav", rate=audio_fs, data=scaled_data)

In [ ]:
# Alternatively, save audio with soundfile or IPython

# from IPython.display import Audio
# import soundfile as sf

# sf.write(f"{paths['analysis']}/{recording}_audio.wav", 
#          scaled_data, audio_fs, subtype='PCM_16')

# You can also try to converty directly the time-series data
# Audio(y_variable, rate=audio_fs)

# Animation: merge audio and video

Merge audio and video using the library [MoviePy](https://zulko.github.io/moviepy/)

In [ ]:
%%time

# Load the video and audio files
video_clip = VideoFileClip(f"{paths['analysis']}/{recording}_video.mp4")
audio_clip = AudioFileClip(f"{paths['analysis']}/{recording}_audio.wav")

# Set the audio of the video clip
video_clip = video_clip.set_audio(audio_clip)

# Save the video
# save_path = f'path/to/file/{experiment_id}_merge.mp4'
save_path = f"{paths['analysis']}/{recording}_merge.mp4"

video_clip.write_videofile(save_path, 
                           codec='libx264', bitrate='3000k',
                           audio_codec='aac', temp_audiofile='temp_audio.m4a', 
                           remove_temp=True)

# Close the clips
video_clip.close()
audio_clip.close()